In [1]:
import numpy as np
import pandas as pd

train = pd.read_csv("./dataset/Traffic accident info/Traffic accident info(12.1 ~ 17.6).csv", encoding='ms949')
day_weather = pd.read_csv("./dataset/subdata/11. Observe daily weather in several district/observe daily weather(2012~2017).csv", encoding='ms949')

# '발생년', '사고유형', '법규위반_대분류', '도로형태', '당사자종별_1당', '당사자종별_2당'
for idx in [22,20,18,15,14, 0]:
    train = train.drop(columns=train.columns[idx])
    
train_col = train.columns
day_weather_col = day_weather.columns

In [2]:
# Get 발생지시도 - 서울 dataset 
train_array = np.array(train)
seoul_dataset = []

for i in range(len(train_array)):
    if train_array[i,9] == '서울':
        seoul_dataset.append(train_array[i])
        
seoul_dataset = np.array(seoul_dataset)

day_order = seoul_dataset[:,0]
sorted_order = sorted(range(len(day_order)), key=lambda k: day_order[k])
seoul_dataset=seoul_dataset[[sorted_order]]

# check result
print(len(seoul_dataset),list(train['발생지시도']).count('서울'))

2050 2050


In [3]:
# make day_weather DataFrame to dict
uniq_days = list(map(str,set(day_weather[day_weather.columns[0]])))
uniq_regions = list(set(day_weather[day_weather.columns[1]]))

day_weather_dict = dict.fromkeys(sorted(uniq_days))

for day in uniq_days:
    day_weather_dict[day] = dict.fromkeys(sorted(uniq_regions))
    for rg in uniq_regions:
        day_weather_dict[day][rg] = list()

for day in np.array(day_weather):
    day_weather_dict[str(day[0])][day[1]].append(day[2:])

In [4]:
# Concat Seoul Traffic Dataset with Seoul Day Weather
seoul_dataset = list(seoul_dataset)
error_indices = []

# try concat but except when missing data or key values error has been raised"
for i in range(len(seoul_dataset)):
    rg = '중구' if seoul_dataset[i][10] == '중구' else seoul_dataset[i][10][:-1]
    try :
        if not day_weather_dict[str(seoul_dataset[i][0])[:8]][rg] :
            error_indices.append(i)
        else :
            seoul_dataset[i] = np.concatenate((seoul_dataset[i],
                                               day_weather_dict[str(seoul_dataset[i][0])[:8]][rg][0]))
    except : 
        error_indices.append(i)

# Remove Error rows by missing data
for i in list(reversed(error_indices)):
    seoul_dataset.pop(i)
    
seoul_dataset = np.array(seoul_dataset)

In [5]:
# Make processed Dataset to csv file without one-hot encoding
seoul_col = train_col.append(day_weather_col[2:])
seoul_dataset_pd = pd.DataFrame(seoul_dataset, columns=seoul_col)
seoul_dataset_pd.to_csv("./concat_ver1.csv",index=None,encoding='ms949')

In [6]:
# get columns' names that have string values For one-hot encoding
str_col = []
for i, w in enumerate(seoul_dataset[0]):
    if type(w) is str:
        str_col.append(i)

In [7]:
# get
uniq_class = []

for idx in str_col:
    uniq_class.append(list(sorted(set(seoul_dataset[:,idx]))))

In [8]:
for idx in range(len(seoul_dataset)):
    for i, col in enumerate(str_col):
        seoul_dataset[idx,col] = int(uniq_class[i].index(seoul_dataset[idx,col]))
        
seoul_dataset_pd = pd.DataFrame(seoul_dataset, columns=seoul_col)
seoul_dataset_pd.to_csv("./concat_ver2.csv",index=None,encoding='ms949')